Laden der Daten von Kaggle und die eigene `no_flood_images`

In [ ]:
import kagglehub
from pathlib import Path
import cv2

# Download 
dataset_dir = Path(kagglehub.dataset_download("faizalkarim/flood-area-segmentation"))
mask_dir = dataset_dir / "Mask"
image_dir = dataset_dir / "Image"

# Output structure
base_dir = Path.home() / "christina" / "dataset" / "yolo" / "augmented"
img_out_dir = base_dir / "images"
mask_out_dir = base_dir / "masks"
label_out_dir = base_dir / "labels"

img_out_dir.mkdir(parents=True, exist_ok=True)
mask_out_dir.mkdir(parents=True, exist_ok=True)
label_out_dir.mkdir(parents=True, exist_ok=True)

# Load original data
image_paths = sorted(image_dir.glob("*.jpg"))
mask_paths = sorted(mask_dir.glob("*.png"))
images = [cv2.imread(str(p)) for p in image_paths]
masks  = [cv2.imread(str(p), cv2.IMREAD_GRAYSCALE) for p in mask_paths]


100%|██████████| 107M/107M [00:03<00:00, 30.2MB/s] 

Extracting files...


### Data augmentation

Die Daten wurde erweitert durch folgende techniken:
- flipping
- cropping
- rotation
- brightness adjustment


Ich habe dafür die [Albumentations](https://albumentations.ai/docs/) Bibliothek verwendet.

Ich hab mich gegen "coarse dropout" entschieden, weil es unrealistsch ist, denke ich. Man bekommt nicht ein Foto, an dem ein Teil fehlt. Aber vielleicht, ist es Sinnvoll.

#### Für die Unterscheidung von Himmel und Wasser zusätzlich noch ergänzt:

Also um mit überbelichteten und kontrastarmen Bildern besser umzugehen:
- `gamma_aug`: Gamma-Korrektur RandomGamma(gamma_limit=(80, 120))

RandomGamma verstärkt und verändert Kontrast und Helligkeit auf nicht-lineare Weise.


In [12]:
import albumentations as A
import os
from pathlib import Path
import cv2
import numpy as np


def augment(images, masks):
    augmented_images = []
    augmented_masks = []

    for img, msk in zip(images, masks):
        augmented_images.append(img)
        augmented_masks.append(msk)

        transforms = [
            A.Resize(height=224, width=224, p=1.0),
            A.GridDistortion(p=1.0, distort_limit=0.6),
            A.OpticalDistortion(p=1.0, distort_limit=1, mode='camera'),
            A.ElasticTransform(p=1.0, alpha=120, sigma=120),
            A.HorizontalFlip(p=1.0),
            A.RandomRotate90(p=1.0),
            A.RandomBrightnessContrast(p=1.0),
            A.GaussianBlur(p=1.0, sigma_limit=(1, 1.5)),
            A.RandomGamma(gamma_limit=(80, 120), p=1.0)
        ]

        for transform in transforms:
            result = transform(image=img, mask=msk)
            augmented_images.append(result['image'])
            augmented_masks.append(result['mask'])

    return augmented_images, augmented_masks


valid_pairs = [(img, msk) for img, msk in zip(images, masks) if img is not None and msk is not None]

if not valid_pairs:
    raise ValueError("no valid image-mask pair!")

images, masks = zip(*valid_pairs)

aug_images, aug_masks = augment(images, masks)

# Save images and masks
for i, (img, mask) in enumerate(zip(aug_images, aug_masks)):
    if img is None or mask is None or img.size == 0 or mask.size == 0:
        print(f"Skipped image {i:04d}")
        continue

    img = np.clip(img, 0, 255).astype(np.uint8)
    mask = np.clip(mask, 0, 255).astype(np.uint8)

    try:
        cv2.imwrite(str(img_out_dir / f"img_{i:04d}.jpg"), img)
        cv2.imwrite(str(mask_out_dir / f"img_{i:04d}.png"), mask)
    except Exception as e:
        print(f"Failed to save {i:04d}: {e}")

#### Datenvorbereitung

Create lables from masks -> polygons (contours) in txt


In [13]:
# Generate YOLO polygon labels
mask_files = sorted(mask_out_dir.glob("*.png"))  # augmented + originals masks
for mask_path in mask_files:
    mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

    H, W = mask.shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    polygons = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            polygon = []
            for (x, y) in cnt.squeeze():
                polygon += [x / W, y / H]
            polygons.append(polygon)

    label_path = label_out_dir / f"{mask_path.stem}.txt"
    with label_path.open("w") as f:
        for poly in polygons:
            f.write("0 " + " ".join(map(str, poly)) + "\n")

print(f"Wrote {len(list(label_out_dir.iterdir()))} label files to {label_out_dir}")

Wrote 2890 label files to dataset/yolo/augmented/labels


In [14]:
# show how this created txt labels look like

import random

all_label_files = list(label_out_dir.glob("*.txt"))
random_label = random.choice(all_label_files)
print(f"\n--- content of a random label file {random_label.name} ---")
with random_label.open() as f:
    print(f.read())


--- content of a random label file img_2559.txt ---
0 0.83 0.7781065088757396 0.8291666666666667 0.7795857988165681 0.8291666666666667 0.7810650887573964 0.83 0.7825443786982249 0.83 0.7855029585798816 0.8291666666666667 0.7869822485207101 0.8291666666666667 0.7899408284023669 0.8266666666666667 0.7943786982248521 0.8258333333333333 0.7943786982248521 0.825 0.7958579881656804 0.825 0.8017751479289941 0.8241666666666667 0.8032544378698225 0.8241666666666667 0.8076923076923077 0.8233333333333334 0.8091715976331361 0.8233333333333334 0.8121301775147929 0.8225 0.8136094674556213 0.8225 0.8165680473372781 0.8216666666666667 0.8180473372781065 0.8216666666666667 0.8210059171597633 0.8208333333333333 0.8224852071005917 0.8208333333333333 0.8254437869822485 0.82 0.8269230769230769 0.82 0.8284023668639053 0.8191666666666667 0.8298816568047337 0.8191666666666667 0.8372781065088757 0.815 0.8446745562130178 0.8141666666666667 0.8446745562130178 0.8133333333333334 0.8461538461538461 0.813333333333